In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import keras.backend as K

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import PIL.Image as Image

# Data Loading

In [ ]:
basepath='/content/drive/My Drive/AllImages'
traincsv=pd.read_csv('/content/drive/My Drive/train.csv')
valpath='/content/drive/My Drive/Validation'
valcsv=pd.read_csv('/content/drive/My Drive/dev.csv')

In [ ]:
traincsv['class'] = traincsv['annotation'].isnull().replace({True: 'Empty', False: 'Foreign'})
traincsv = traincsv[:8000]
valcsv['class'] = valcsv['annotation'].isnull().replace({True: 'Empty', False: 'Foreign'})
valcsv = valcsv[:1000]

In [ ]:
print(traincsv)
print(valcsv)

# Setting Model Parameters
Transfer Learning


In [ ]:
BATCH_SIZE = 32 # Number of training examples to process before updating our models variables
IMAGE_RES=224
IMG_SHAPE  = (IMAGE_RES, IMAGE_RES)   # Our training data consists of images with width of 224 pixels and height of 224 pixels

In [ ]:
train_image_generator      = ImageDataGenerator(rescale=1./255)  # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_dataframe(
    traincsv,
    directory=basepath,
    x_col='image_name',
    y_col='class',
    target_size=IMG_SHAPE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=True)
    #save_to_dir='/content/drive/My Drive/Results')

validation_data_gen = validation_image_generator.flow_from_dataframe(
    valcsv,
    directory=valpath,
    x_col='image_name',
    y_col='class',
    target_size=IMG_SHAPE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=True)
    #save_to_dir='/content/drive/My Drive/Results')


In [ ]:
#URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [ ]:
feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

In [ ]:
model.compile( 
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['AUC'])



In [ ]:
###### Test Wirh Different LR and Metric
import keras.backend as K

model= tf.keras.models.load_model(
  path,
  custom_objects={'KerasLayer': hub.KerasLayer})

'''print(K.eval(model.optimizer.lr))
K.set_value(model.optimizer.learning_rate, 0.0005)
print(K.eval(model.metrics))'''



model.compile( 
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['auc'])


In [ ]:
 t=time.time()
path="/content/drive/My Drive/models/edi/{}.h5".format(int(t))
print(path)

model.save(path)

In [ ]:
path="/content/drive/My Drive/models/edi/1593439476.h5"
!ls -ltr "/content/drive/My Drive/models/edi/"

In [ ]:
for i in range(0,1):
  EPOCHS = 5
  model= tf.keras.models.load_model(
  path,
  custom_objects={'KerasLayer': hub.KerasLayer})
  
  model.compile( 
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
  
  K.set_value(model.optimizer.learning_rate, 0.00001)
  history = model.fit(train_data_gen,
                    epochs=EPOCHS,
                    validation_data=validation_data_gen)
  
  t=time.time()
  path="/content/drive/My Drive/models/edi/{}.h5".format(int(t))
  model.save(path)


In [ ]:
count=0
for i in range (0,1000):
  pt=valpath+'/'+valcsv["image_name"].iloc[i]

  img=tf.keras.preprocessing.image.load_img(
    pt, grayscale=False, color_mode='rgb', target_size=None,
    interpolation='nearest')
  img=img.resize((IMAGE_RES, IMAGE_RES))
  img = np.array(img)/255.0
  img.shape
  result = model.predict(img[np.newaxis, ...])
  result.shape

  predicted_class = np.argmax(result[0], axis=-1)
  predicted_class
  if ((predicted_class==0 and valcsv["class"].iloc[i]=="Empty") or  (predicted_class==1 and valcsv["class"].iloc[i]=="Foreign")):
    count+=1

print (count)


In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
0plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
print(history.history)